In [0]:
#Using a python wrapper to obtain dataset
from sodapy import Socrata
import calendar
import json
import pandas as pd
from collections import defaultdict
import string

In [0]:
data_source = "data.sfgov.org"
client = Socrata(data_source, None)

In [0]:
#MORE_INFO: https://dev.socrata.com/foundry/data.sfgov.org/5m6g-bqm4
def get_write_data(file_path, api_key,  month_name):
    raw_data = client.get(api_key, month= month_name)
    
    raw_json=json.dumps(raw_data)
    
    #strip array brackets
    raw_json = raw_json.strip("[")
    raw_json = raw_json.strip("]")
    
    with open(file_path, "a") as write_file:
        if month_name == "January":
            write_file.write("[")
        
        write_file.write(raw_json)
        
        if month_name != "DECEMBER":
            write_file.write(",")
        else:
            write_file.write("]")
    write_file.close()    

In [0]:
api_key = "5m6g-bqm4"

months = []
for month in calendar.month_name[1:]:
    months.append(month)
    months.append(month.upper())
    
file_path="C:/Users/Wollyz/Documents/Computer Science 2/Data Science in Python/noise_rating.json"

for m in months:
    get_write_data(file_path, api_key, m)

In [0]:
f1=open("C:/Users/Wollyz/Documents/Computer Science 2/Data Science in Python/noise_rating.json","r")
data1 = json.loads(f1.read())
f1.close()

In [0]:
#convert to a frame
frame = defaultdict(list)
empty_idx = []
for data in data1:
    #missing values for airline name
    if len(data.keys()) < 9:
        frame["airline"].append("EMPTY")
    for key in data:
        if key == "airline":
            if data[key] == "ERROR: #N/A":
                break
        frame[key].append(data[key])

In [0]:
length_dict = {key:len(value) for key, value in frame.items()}

In [0]:
#convert airlines with names in lower caps to all upper caps
new_airline = []
for i in frame["airline"]:
    val = i
    if i.isupper() == False:
        val = i.upper()
    new_airline.append(val)

v_index = [i for i, x in enumerate(new_airline) if x == "VOLGA\xadDNEPR"]

for i in v_index:
    new_airline[i] = "VOLGA-DNEPR"
    
frame["airline"] = new_airline

unique_airline = []
unique_code = []

for i, x in enumerate(frame["airline"]):
    if x not in unique_airline and x != "EMPTY":
        unique_airline.append(x)
        unique_code.append(frame["airline_code"][i])

In [0]:
#get the rows where the airline field is empty i.e. labelled "EMPTY"
empty_idx = [i for i, x in enumerate(new_airline) if x == "EMPTY"]
code_idx = []
retrieved_name =[]
for idx in empty_idx:
    code = frame["airline_code"][idx]
    if code in unique_code:
        match = unique_code.index(code)
        ins = idx, unique_airline[match]
        retrieved_name.append(ins)
        code_idx.append(idx)

#update the names retrieved, 
for idx, name in retrieved_name:
    frame["airline"][idx] = name

#delete the ones that were not as there is not enough examples
discard_idx = [i for i in empty_idx if i not in code_idx]
columns = frame.keys()
while("EMPTY" in frame["airline"]):
    next_empty = frame["airline"].index("EMPTY")
    for col in columns:
        del frame[col][next_empty]
    

In [0]:
#check if they are airlines with the same aircode but with additional name eg. ETIHAD and ETIHAD AIRWAYS
other_idx = []
for i in range(len(unique_code)):
    code = unique_code[i]
    
    #get the index where there are more than occureences of the airline code 
    occ_idx = [j for j, x in enumerate(unique_code) if x == code]
    
    if len(occ_idx) > 1:
        cnt = 0
        for o in other_idx:
            if i in o:
                cnt += 1
        if cnt == 0:
            other_idx.append(occ_idx)

In [0]:
for org, other in other_idx:
    dup_name = unique_airline[other]
    org_name = unique_airline[org]
    #get the indexes in frame dict. with dup_name
    names_idx = [j for j, x in enumerate(frame["airline"]) if x == dup_name]
    for idx in names_idx:
        frame["airline"][idx] = org_name

In [0]:
frame_init = {"airline": pd.Series(frame["airline"], dtype="str"),\
              "airline_code": pd.Series(frame["airline_code"], dtype="str"),\
              "exceedances_per_1000_operations": pd.Series(frame["exceedances_per_1000_operations"], dtype="int"),\
              "month": pd.Series(frame["month"], dtype="str"),\
              "noise_exceedance_quality_rating_score": pd.Series(frame["noise_exceedance_quality_rating_score"], dtype="float"),\
              "noise_exceedance_rating": pd.Series(frame["noise_exceedance_rating"], dtype="float"),\
              "total_noise_exceedances": pd.Series(frame["total_noise_exceedances"], dtype="int"),\
              "total_operations_per_month": pd.Series(frame["total_operations_per_month"], dtype="int"),\
              "year": pd.Series(frame["year"], dtype="str")}

rating_table = pd.DataFrame(frame_init)

In [0]:
rating_table[rating_table["airline"]== "EMPTY"]

,airline,airline_code,exceedances_per_1000_operations,month,noise_exceedance_quality_rating_score,noise_exceedance_rating,total_noise_exceedances,total_operations_per_month,year


In [0]:
months_format = calendar.month_name[1:]

In [0]:
#modify e.g. 'APRIL' to 'April'
for m in months_format:
    month_idx = rating_table.index[rating_table["month"] == m.upper()]
    
    if len(month_idx > 0):
        for idx in month_idx:
            rating_table.at[idx, "month"] = m

In [0]:
#remove "noise_exceeding_rating"
redun_feature = "noise_exceedance_rating"
cols = [i for i in rating_table.columns if i not in redun_feature]
rating_table = rating_table[cols]

In [0]:
#check for missing values
rating_table.isnull().sum()

airline                                  0
airline_code                             0
exceedances_per_1000_operations          0
month                                    0
noise_exceedance_quality_rating_score    0
total_noise_exceedances                  0
total_operations_per_month               0
year                                     0
dtype: int64

In [0]:
simplify_table = rating_table[rating_table.columns.difference(["month"])]
yearly = simplify_table.groupby(["airline", "airline_code", "year"])
yearly.mean()

exceedances_per_1000_operations  \
airline               airline_code year                                    
ACM AVIATION          BJT          2009                        83.000000   
AER LINGUS            EIN          2008                        27.000000   
                                   2009                        29.000000   
                                   2014                        24.000000   
                                   2015                        23.000000   
                                   2017                        18.000000   
                                   2018                        42.000000   
AEROMEXICO            AMX          2009                       120.818182   
                                   2010                       133.916667   
                                   2011                        91.666667   
                                   2012                       112.083333   
                                   2013                       104.166667   
                                   2014                        56.916667   
                                   2015                        49.583333   
                                   2016                       149.083333   
                                   2017                       203.750000   
                                   2018                       181.750000   
AEROUNION             TNO          2009                      1000.000000   
AIR BERLIN            BER          2011                       125.000000   
                                   2012                        38.000000   
                                   2016                        23.000000   
                                   2017                       103.000000   
AIR CANADA            ACA          2006                        15.916667   
                                   2007                        20.916667   
                                   2008                        12.750000   
                                   2009                        22.454545   
                                   2010                        15.666667   
                                   2011                        20.583333   
                                   2012                        25.916667   
                                   2013                        27.083333   
...                                                                  ...   
VOLARIS               VOI          2016                       103.000000   
                                   2017                        36.500000   
                                   2018                       149.545455   
VOLGA-DNEPR           VDA          2007                       500.000000   
                                   2017                       375.000000   
                                   2018                      1000.000000   
WESTJET AIRLINES      WJA          2006                       125.000000   
                                   2009                       158.500000   
                                   2010                       253.500000   
                                   2011                         8.000000   
                                   2012                         8.000000   
                                   2013                        36.500000   
                                   2014                        38.500000   
                                   2015                        46.200000   
                                   2016                       106.166667   
                                   2017                        66.200000   
                                   2018                        22.400000   
WORLD AIRWAYS         WOA          2010                      2987.600000   
                                   2011                      1764.750000   
                                   2012                      1774.700000   
WORLDWIDE JET

In [0]:
#descriptive features
desc_table = rating_table[rating_table.columns.difference(["month", "year", "airline", "airline_code"])]
desc_table.describe()

,exceedances_per_1000_operations,noise_exceedance_quality_rating_score,total_noise_exceedances,total_operations_per_month
count,5965.000000,5965.000000,5965.000000,5965.000000
mean,318.879631,8.532117,42.713998,810.912154
std,556.745062,2.427520,73.684081,1900.327467
min,1.000000,0.000000,1.000000,1.000000
25%,21.000000,8.330000,5.000000,62.000000
50%,52.000000,9.740000,17.000000,120.000000
75%,356.000000,9.910000,49.000000,581.000000
max,4500.000000,10.000000,815.000000,14016.000000
